In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

In [3]:
from ln_utils import *

In [4]:
%matplotlib inline

# Parameters

In [5]:
is_directed = True
time_window = 86400*7

# Load temporal data

In [6]:
graph_files = []

In [7]:
data_dir = "../LNdata/snapshots/"
graph_files +=  [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f][:49] # data files in the second part are corrupted???
MIN_TIME = 1539302399 #Thursday, October 11, 2018 11:59:59 PM
#MAX_TIME = 1553126399 #Wednesday, March 20, 2019 11:59:59 PM

In [8]:
data_dir = "../LNdata/lncaptures/lngraph/2019/"
graph_files +=  [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
#MIN_TIME = 1549065601-86400 #Saturday, February 2, 2019 12:00:01 AM
#MAX_TIME = 1552867201 #Monday, March 18, 2019 12:00:01 AM

In [9]:
data_dir = "../LNdata/"
#graph_files = [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
graph_files += [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f][5:]
#MIN_TIME = 1552478399 # Wednesday, March 13, 2019 11:59:59 AM
MAX_TIME = 1553947199 # Saturday, March 30, 2019 11:59:59 AM

graph_files

In [10]:
nodes, edges = load_temp_data(graph_files[:-1])
print(len(nodes), len(edges))

../LNdata/snapshots/lngraph_2018_10_13__12_00.json 1636 11499
../LNdata/snapshots/lngraph_2018_10_14__12_00.json 1804 16710
../LNdata/snapshots/lngraph_2018_10_15__12_00.json 1966 21864
../LNdata/snapshots/lngraph_2018_10_16__12_00.json 2168 27063
../LNdata/snapshots/lngraph_2018_10_17__12_00.json 2348 32268
../LNdata/snapshots/lngraph_2018_10_18__12_00.json 2489 37555
../LNdata/snapshots/lngraph_2018_10_19__12_00.json 2704 42956
../LNdata/snapshots/lngraph_2018_10_20__12_00.json 2941 48452
../LNdata/snapshots/lngraph_2018_10_21__12_00.json 3128 53655
../LNdata/snapshots/lngraph_2018_10_22__12_00.json 3267 58954
../LNdata/snapshots/lngraph_2018_10_23__12_00.json 3442 64371
../LNdata/snapshots/lngraph_2018_10_24__12_00.json 3651 69779
../LNdata/snapshots/lngraph_2018_10_25__12_00.json 3847 75388
../LNdata/snapshots/lngraph_2018_10_26__12_00.json 4032 80861
../LNdata/snapshots/lngraph_2018_10_27__12_00.json 4202 86248
../LNdata/snapshots/lngraph_2018_10_28__12_00.json 4387 91592
../LNdat

# I DO NOT UNDERSTAND WHY THERE IS A GAP!!!???

 * there can be bugs in the data? Due to MIN_TIME MAX_TIME filter the edges disappeared from the missing time interval?

edges_lower_cut = edges[(edges["last_update"] > MIN_TIME)]

edges_upper_cut = edges_lower_cut[(edges_lower_cut["last_update"] < MAX_TIME)]

len(edges_lower_cut), len(edges_upper_cut)

edges_upper_cut["last_update"].hist()

In [24]:
nodes = nodes[(nodes["last_update"] > MIN_TIME) & (nodes["last_update"] < MAX_TIME)]
edges = edges[(edges["last_update"] > MIN_TIME) & (edges["last_update"] < MAX_TIME)]
len(nodes), len(edges)

(49802, 1736790)

In [25]:
edges = edges.sort_values("last_update")

#### Capacity on edges can change

In [26]:
edges.groupby(["node1_pub","node2_pub"])["capacity"].nunique().value_counts()

1     52883
2      4717
3       559
4       120
5        37
6        21
7         5
12        1
11        1
9         1
8         1
32        1
Name: capacity, dtype: int64

# Analyzing graph snapshots

### Snapshots with the number of edges

In [28]:
snaphots = get_snapshots(edges, MIN_TIME, MAX_TIME, with_data=True, time_window=time_window)

0 36553
1 38236
2 37857
3 36567
4 68
5 51
6 89
7 58
8 98
9 130
10 131
11 335
12 13951
13 66004
14 78206
15 63876
16 130905
17 141290
18 156094
19 165494
20 173476
21 193935
22 196787
23 197232


In [27]:
alma

NameError: name 'alma' is not defined

### Calculate centralities for snapshots

stats = get_snapshot_properties(snaphots, is_directed=is_directed, weight="capacity")

In [ ]:
stats = get_snapshot_properties(snaphots, is_directed=is_directed, weight=None)

In [ ]:
for idx, snap_stat in enumerate(stats):
    snap_stat.to_csv("stat_%i.csv" % idx, index=False)

# Correlation time series

In [ ]:
results = dict([(corr, get_corr_sequence(stats, corr)) for corr in ["pearson","spearman","kendall","w_kendall"]])

### Correlation of centrality measures on adjacent days

In [ ]:
for cent in  ["deg","in_deg","out_deg","pr","betw","harm"]:
    show_corr_results(results, cent)